Python:modelado y ETL
Implementación del diseño del datamart mediante un script en python, usando librería pandas para el manejo de los dataframes.

In [6]:
#Instalación de librerías
!pip install pandas
!pip install numpy

In [12]:
#Cargamos y exploramos datos
import pandas as pd

invoices_header = pd.read_csv('DatosProyecto/invoices_header.csv')
invoices_products = pd.read_csv('DatosProyecto/invoices_products.csv')
products = pd.read_csv('DatosProyecto/products.csv')
suppliers = pd.read_csv('DatosProyecto/suppliers.csv')
daily_currencies = pd.read_csv('DatosProyecto/daily_currencies.csv')

# Muestra las primeras filas de los datos
invoices_header.head(), suppliers.head()


(  Invoice;"InboundDate";"Supplier";"OrderDate";"InvoiceDate"
 0  FFCC141196;"2014-05-26";"PROV1650";"2014-05-10...        
 1  FFCC141197;"2014-05-20";"PROV40000235";"2014-0...        
 2  FFCC141198;"2014-05-12";"PROV1647";"2014-04-27...        
 3  FFCC141199;"2014-05-19";"PROV40001000";"2014-0...        
 4  FFCC141200;"2014-05-24";"PROV40000850";"2014-0...        ,
   SupplierName;"PaymentMethod";"PaymentTerms";"IDSupplier";"Country";"Currency"
 0  Proveedor 3;"RECIBO";"1X30";"PROV41000270";"ES...                           
 1  Proveedor 4;"RECIBO";"1X60";"PROV1344";"ES";"EUR"                           
 2  Proveedor 12;"RECIBO";"1X85";"PROV40000010";"E...                           
 3  Proveedor 13;"RECIBO";"1X60";"PROV40000187";"E...                           
 4  Proveedor 14;"RECIBO";"1X60";"PROV40000200";"E...                           )

Comprobación de los tipos de datos

In [16]:
print(invoices_header.dtypes)
print(invoices_products.dtypes)
print(products.dtypes)
print(suppliers.dtypes)
print(daily_currencies.dtypes)

Invoice;"InboundDate";"Supplier";"OrderDate";"InvoiceDate"    object
dtype: object
Invoice;"Quantity";"Product";"PurchasePrice (Unit)";"Section"    object
dtype: object
Product;"Type";"Division";"Group";"ShortDescription";"Description"    object
dtype: object
SupplierName;"PaymentMethod";"PaymentTerms";"IDSupplier";"Country";"Currency"    object
dtype: object
Date;"Open";"High";"Low";"Close";"Currency"    object
dtype: object


Los archivos usan punto y coma en vez de coma. Vamos a especificar el delimitador y convertimos las columnas a los tipos adecuados. 

In [20]:
invoice_header = pd.read_csv('DatosProyecto/invoices_header.csv', sep=';')
invoice_products = pd.read_csv('DatosProyecto/invoices_products.csv', sep=';')
products = pd.read_csv('DatosProyecto/products.csv', sep=';')
suppliers = pd.read_csv('DatosProyecto/suppliers.csv', sep=';')
daily_currencies = pd.read_csv('DatosProyecto/daily_currencies.csv', sep=';')

#Verificamos los tipos de datos después de haber corregido el delimitador
print(invoice_header.dtypes)
print(invoice_products.dtypes)
print(products.dtypes)
print(suppliers.dtypes)
print(daily_currencies.dtypes)


Invoice        object
InboundDate    object
Supplier       object
OrderDate      object
InvoiceDate    object
dtype: object
Invoice                  object
Quantity                  int64
Product                  object
PurchasePrice (Unit)    float64
Section                  object
dtype: object
Product             object
Type                object
Division            object
Group               object
ShortDescription    object
Description         object
dtype: object
SupplierName     object
PaymentMethod    object
PaymentTerms     object
IDSupplier       object
Country          object
Currency         object
dtype: object
Date         object
Open        float64
High        float64
Low         float64
Close       float64
Currency     object
dtype: object


Convertimos la columna de fecha a datetime:

In [23]:
# Convierte las columnas de fechas a tipo datetime
invoice_header['InboundDate'] = pd.to_datetime(invoice_header['InboundDate'], format='%Y-%m-%d')
invoice_header['OrderDate'] = pd.to_datetime(invoice_header['OrderDate'], format='%Y-%m-%d')
invoice_header['InvoiceDate'] = pd.to_datetime(invoice_header['InvoiceDate'], format='%Y-%m-%d')


Comprobamos si hay valores nulos 

In [26]:
invoice_header.isnull().sum()

Invoice        0
InboundDate    0
Supplier       0
OrderDate      0
InvoiceDate    0
dtype: int64

Revisamos que no haya valores nulos en los demas dataframes:

In [39]:
print("Valores nulos en invoice_products:")
print(invoice_products.isnull().sum())

print("Valores nulos en products:")
print(products.isnull().sum())

print("Valores nulos en suppliers:")
print(suppliers.isnull().sum())

print("Valores nulos en daily_currencies:")
print(daily_currencies.isnull().sum())

Valores nulos en invoice_products:
Invoice                 0
Quantity                0
Product                 0
PurchasePrice (Unit)    0
Section                 0
dtype: int64
Valores nulos en products:
Product                1
Type                3393
Division              21
Group                 19
ShortDescription     103
Description            0
dtype: int64
Valores nulos en suppliers:
SupplierName     0
PaymentMethod    7
PaymentTerms     6
IDSupplier       0
Country          0
Currency         0
dtype: int64
Valores nulos en daily_currencies:
Date        0
Open        0
High        0
Low         0
Close       0
Currency    0
dtype: int64


La columna Product tiene 1 valor nulo. En este caso lo eliminamos. La columna Type tiene muchos valores nulos 3393. Podemos rellenarlos con un valor como 'Desconocido' en lugar de eliminarlos todos. Lo mismo haremos con los valores nulos encontrados en Division, Group, PaymentMethod, PaymentTerms y ShortDescription. 

In [43]:
#Rellenamos los nulos en products
products = products.fillna({
    'Type': "Desconocido",
    'Division': "Desconocido",
    'Group': "Desconocido",
    'ShortDescription': "Desconocido"
})

#Eliminamos la fila donde falta el ID de producto
products = products.dropna(subset=['Product'])

#Rellenamos nulos en suppliers

suppliers = suppliers.fillna({
    'PaymentMethod': "No especificado",
    'PaymentTerms': "No especificado"
})

Verificamos los tipos de datos después de rellenar los valores nulos:

In [45]:
print(products.dtypes)
print(suppliers.dtypes)
print(invoice_header.dtypes)
print(invoice_products.dtypes)
print(daily_currencies.dtypes)

Product             object
Type                object
Division            object
Group               object
ShortDescription    object
Description         object
dtype: object
SupplierName     object
PaymentMethod    object
PaymentTerms     object
IDSupplier       object
Country          object
Currency         object
dtype: object
Invoice                   object
InboundDate       datetime64[ns]
Supplier                  object
OrderDate         datetime64[ns]
InvoiceDate       datetime64[ns]
lead_time_real             int64
dtype: object
Invoice                  object
Quantity                  int64
Product                  object
PurchasePrice (Unit)    float64
Section                  object
dtype: object
Date         object
Open        float64
High        float64
Low         float64
Close       float64
Currency     object
dtype: object


Calculamos el Lead Time como la diferencia entre la fecha de recepción y la de pedido

In [29]:
invoice_header['lead_time_real'] = (invoice_header['InboundDate'] - invoice_header['OrderDate']).dt.days


In [47]:
print(invoice_header[['InboundDate', 'OrderDate', 'lead_time_real']].head(10))

  InboundDate  OrderDate  lead_time_real
0  2014-05-26 2014-05-10              16
1  2014-05-20 2014-04-13              37
2  2014-05-12 2014-04-27              15
3  2014-05-19 2014-03-24              56
4  2014-05-24 2014-05-11              13
5  2014-05-15 2014-04-03              42
6  2014-05-30 2014-04-15              45
7  2014-05-26 2014-05-05              21
8  2014-05-27 2014-04-20              37
9  2014-05-01 2014-04-06              25


De esta forma hemos comprobado si había valores nulos, limpiado los nombres de las columnas y convertido a formato datetime las fechas y calculado el Lead Time.

Por último, guardamos los dataframes en archivos csv transformados

In [33]:
invoice_header.to_csv('Invoice_header_transformed.csv', index=False)
invoice_products.to_csv('Invoice_products_transformed.csv', index=False)
products.to_csv('Products_transformed.csv', index=False)
suppliers.to_csv('Suppliers_transformed.csv', index=False)
daily_currencies.to_csv('Daily_currencies_transformed.csv', index=False)

Verificamos que los archivos se han guardado correctamente:

In [36]:
check_df = pd.read_csv('Invoice_header_transformed.csv')
print(check_df.head())

      Invoice InboundDate      Supplier   OrderDate InvoiceDate  \
0  FFCC141196  2014-05-26      PROV1650  2014-05-10  2014-05-29   
1  FFCC141197  2014-05-20  PROV40000235  2014-04-13  2014-05-23   
2  FFCC141198  2014-05-12      PROV1647  2014-04-27  2014-05-15   
3  FFCC141199  2014-05-19  PROV40001000  2014-03-24  2014-05-22   
4  FFCC141200  2014-05-24  PROV40000850  2014-05-11  2014-05-27   

   lead_time_real  
0              16  
1              37  
2              15  
3              56  
4              13  


El archivo se ha guardado correctamente y los datos se ven bien estructurados. 